<b><font size=20, color='#A020F0'>Satpy</font></b>

Hannah Zanowski<br>
12/2/24<br>

#### <span style="color:green">Learning Goals</span>
By the end of this notebook you will
1. Become familiar with the basic data structures in satpy
2. Use satpy to read in and plot images of satellite data
3. Practice other basic satellite data manipulation techniques such as compositing and resampling

#### Resources
[Satpy Documentation](https://satpy.readthedocs.io/en/stable/)<br>
[Satpy API reference](https://satpy.readthedocs.io/en/stable/api/modules.html)<br>
[goes2go](https://github.com/blaylockbk/goes2go)

#### Acknowledgements
This lecture draws from examples in the satpy documentation as well as a few provided online by D. Hoese.

# A little about Satpy

Satpy is a package used for processing and manipulating meteorological remote sensing data and writing it to various file formats. Satpy was originally developed by [David Hoese](https://github.com/djhoese) (pronounced 'haze') and crew here in our building at the Space Science and Engineering Center. Like many packages, it uses xarray and dask under the hood.

Satpy is part of [PyTroll](http://pytroll.github.io/), which is a broader python framework for visualizing Earth-observing satellite data! Many members of the PyTroll team have contributed to the development of satpy.

#### <font color='red'><b>BEFORE WE BEGIN WE NEED TO CHANGE OUR DEFAULT ENVIRONMENT</n></font>

Now let's import some of the standard packages that we use (we'll import stuff for Satpy later):

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

---

## 1. GOES data

In today's lecture we'll be working with data from [GOES (Geostationary Operational Environmental Satellites) satellites](https://www.nasa.gov/content/goes-overview/index.html).

There are many different ways to look at GOES data, but here are a couple of neat ones:
1. [NESDIS STAR GOES Image Viewer](https://www.star.nesdis.noaa.gov/goes/index.php)
2. [RAMMB/CIRA Slider](https://rammb-slider.cira.colostate.edu/) (also through NESDIS)

We actually have a couple of GOES receivers on our roof (the big dishes with Christmas lights on them, according to Pete), which seems appropriate for a building that is described as the "birthplace of satellite meteorology."

Data from GOES 16, 17, and 18 are available in a number of places including through [Amazon Web Services](https://registry.opendata.aws/noaa-goes/). You can also easily download GOES data using [goes2go](https://github.com/blaylockbk/goes2go) which is in our environment.

If you're interested, here is a handy set of slides about [working with GOES data](https://www.goes-r.gov/downloads/resources/documents/Beginners_Guide_to_GOES-R_Series_Data.pdf)



---

## 2. Satpy

### 2.1 Scene Objects
The satpy [Scene](https://satpy.readthedocs.io/en/stable/api/satpy.scene.html#module-satpy.scene) class is the main way in which you interact with your satellite data. A satpy Scene is sort of like an xarray Dataset in that it is just a container that holds the data you are interested in. 

You need two pieces of information to set up a satpy Scene:
1. A data reader
2. A set of files to read
    ><b><font color='red'>Note:</font> It is extremely important that the supplied files have kept their standard names  (i.e. the names should be what they were when you accessed/downloaded the data) otherwise the data reader will get upset </b>

Here's how to create a scene from some GOES-18 data (specifically from the [level 1b radiance products](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C01501) from the [Advanced Baseline Imager (ABI)](https://www.goes-r.gov/spacesegment/abi.html):

In [ ]:
from satpy import Scene
from glob import glob

path='/share/Lecture_data/GOES18/'
scn=Scene(reader='abi_l1b', filenames=glob(path+'OR_ABI*.nc'))

Great! But what did we really do? Not all that much (yet). To look at our data and start working with it we'll need to load it, which we'll do after we talk a bit about data readers.

### 2.2 Readers
satpy comes with a bunch of available data readers that are set up to read files in many different formats from many satellites! To see what readers are available, we can use `available_readers()`:

In [ ]:
from satpy import available_readers
available_readers()

When we created our scene, we specified the data reader to be `abi_l1b`, which is the reader we need if we want to be able to interpret level 1b radiance data from the ABI on GOES-18.

### 2.3 Loading data for use

Let's take a look at what "datasets" we have available in our scene. We can do this with [available_dataset_names()](https://satpy.readthedocs.io/en/stable/api/satpy.scene.html#satpy.scene.Scene.available_dataset_names)
><b><font color='red'>Note:</font></b> Datasets in satpy are really the equivalent of xarray DataArrays. The closest equivalent of an xarray Dataset in satpy would be the scene you created.

In [ ]:
channels=scn.available_dataset_names()
print(channels)

You can also get slightly more information beyond the dataset name by using [available_dataset_ids()](https://satpy.readthedocs.io/en/stable/api/satpy.scene.html#satpy.scene.Scene.available_dataset_ids) instead:

In [ ]:
scn.available_dataset_ids()

Now load the channels you want using [scn.load()](https://satpy.readthedocs.io/en/stable/api/satpy.scene.html#satpy.scene.Scene.load).
><b><font color='red'>Note:</font></b> Be _THOUGHTFUL_ about what you read in and load--satellite data can be ENORMOUS--that's why dask is such a critical part of satpy!

In the examples in this lecture the data aren't too big, so we'll load all of the channels so we can make some interesting composites:

In [ ]:
scn.load(channels)

#Another way to choose data is by the wavelength of each channel, e.g.,:
#scn.load([0.47, 0.64]) #channel wavelengths that you want

You can also filter the channels you load by their calibration, resolution, etc:

In [ ]:
scn.load(channels, calibration='radiance')

You can access the channels that you loaded like you would access specific variables in an xarray dataset:

#### Accessing measured values
You probably noticed above that your data are automatically read in as dask arrays (we will learn more about dask in a few weeks). That means you won't be able to access the actual values of your data unless you either trigger a computation on the data (this can even be something like plotting), or if you append `.load()` to the data your are interested in. Another quick way to load your data into memory is by using `.values`, which we'll do below:

><b><font color='red'>Note:</font></b> Be careful any of the above options will load all your data into memory! If you have a huge amount of data that will be a problem!

In [ ]:
scn['C01'].values

### 2.4 Basic visualization

You can look at any of the channels from the ABI by using [scn.show()](https://satpy.readthedocs.io/en/stable/api/satpy.scene.html#satpy.scene.Scene.show). You can also use [scn.images()](https://satpy.readthedocs.io/en/stable/api/satpy.scene.html#satpy.scene.Scene.images) to generate images from all the datasets in your scene, but we won't do that here.

In [ ]:
scn.show('C08')

Another option is to use matplotlib plotting commands like `pcolormesh()` or [imshow()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html#matplotlib-pyplot-imshow)

In [ ]:
plt.imshow(scn['C08'])
plt.colorbar(orientation='horizontal')

#### Plotting with Cartopy

You can plot your GOES data with cartopy as well. To help with this process, there is a handy function included in satpy called `to_cartopy_crs()` that reads the attributes of your specified data and turns that information into a cartopy crs!

In [ ]:
crs=scn['C01'].attrs['area'].to_cartopy_crs()
crs

In [ ]:
#Set up the plot
fig,ax=plt.subplots(subplot_kw={'projection':crs})
#add coastlines
ax.coastlines()
ax.set_global()
#make the plot using imshow
plt.imshow(scn['C08'], transform=crs, extent=crs.bounds)
cbar=plt.colorbar(orientation='horizontal')
cbar.set_label('Radiance (K)')

### 2.5 Resampling
Resampling is useful when you want to compile data across channels that have varying resolution, like our GOES data!
There are many [resampling options](https://satpy.readthedocs.io/en/stable/resample.html) available in satpy and they are typically used to resample all of your datasets onto a common area that you specify. 

A useful resampler is the 'native' resampler, which will allow you to resample your datasets onto the original or 'native' projection of the datasets.

><b><font color='blue'>Note:</font></b> The default behavior of the native sampler is to sample onto the highest resolution _area_ across the datasets in your scene!

First, let's figure out what the resolutions (i.e., we need shape of the datasets) are of our various channels:

In [ ]:
for channel in channels:
    print(channel, ':', scn[channel].shape)

Now let's resample our datasets, but we'll tell the resampler to resample to the coarsest area instead of the finest:

In [ ]:
scn_resampled=scn.resample(scn.coarsest_area(), resampler='native') #lowest res area

Check that that did what we think it did:

In [ ]:
for channel in channels:
    print(channel, ':', scn_resampled[channel].shape)

### 2.6 Composites
You can make composite images by combining the data from multiple channels. To figure out what kinds of composites you can make in your scene you can use [available_composite_names()](https://satpy.readthedocs.io/en/stable/api/satpy.scene.html#satpy.scene.Scene.available_composite_names):

In [ ]:
scn.available_composite_names()

Let's try to load and plot the 'true_color' composite:

In [ ]:
scn.load(['true_color'])

Hmmmm, what happened? As I alluded to during the resampling and as the warning message above notes, if we want to make a composite, the datasets need to have the same areal resolution. So let's try to generate the composite using our resampled datasets instead:

In [ ]:
scn_resampled.load(['true_color'])

#You can load multiple composites at once
#scn_resampled.load(['true_color','convection'])

Great! Now let's make a plot:

In [ ]:
scn_resampled.show('true_color')

#### Try it!
Take a moment to load a different composite from the list above and make an image of it

### 2.7 Saving output
You can save your datasets in a number of different file formats. You will need to specify a data writer to do so. To see what [data writers](https://satpy.readthedocs.io/en/stable/writers.html#writers) are available in satpy use `available_writers()`

><b><font color='blue'>Note:</font></b> The default writer is geotiff

In [ ]:
from satpy import available_writers
available_writers()

You can save datasets using the [save_datasets](https://satpy.readthedocs.io/en/stable/api/satpy.scene.html#satpy.scene.Scene.save_datasets) method:

In [ ]:
scn_resampled.save_datasets(datasets=['true_color'],filename='GOES18_resampled_truecolor.tiff')

### 2.8 MultiScene
[MultiScene objects](https://satpy.readthedocs.io/en/stable/multiscene.html#multiscene-experimental) are useful when you want to compile data from multiple times/multiple passes of a satellite, data from multiple satellites, or multiple products from the same satellite.

><b><font color='red'>WARNING:</font></b> The MultiScene class is still in its infancy. That means it might not always be able to do what you want. Nevertheless, the functionality seems really cool, and it will be neat to see how this part of satpy develops as time passes.

Before we use MultiScene, let's take a look at the data we'll be working with. It's data from GOES-16 Channel 2 (visible) on 28 Sept 2022, around the time that Hurricane Ian was making landfall in Florida. We'll read in one timestep of the data:

In [ ]:
abi_path='/share/Lecture_data/GOES16/abi/'
scn2=Scene(filenames=[abi_path+'OR_ABI-L1b-RadM1-M6C02_G16_s20222712000282_e20222712000339_c20222712000363.nc'],
           reader='abi_l1b')
scn2.load(['C02'])
scn2.show('C02')

There are two ways to read in files using MultiScene:
1. You can create each scene separately and store them in a list that you provide to MultiScene
>`MultiScene([scene1, scene2, scene3, ...])`
2. Or you can use the [from_files()](https://satpy.readthedocs.io/en/stable/api/satpy.multiscene.html#satpy.multiscene.MultiScene.from_files) method that is a part of MultiScene:

Below we'll use MultiScene to combine the GOES-16 Channel 02 data that we have using option 2:

In [ ]:
from satpy import MultiScene

In [ ]:
abi_path='/share/Lecture_data/GOES16/abi/'
mscn=MultiScene.from_files(glob(abi_path+'OR*-RadM1*G16*.nc'),reader='abi_l1b')
mscn.load(['C02'])

Remember that like Scene, MultiScene is just a container, so if you want to actually be able to see your data like an xarray Dataset, it's not particularly straightforward. One way to do this is to use [blend()](https://satpy.readthedocs.io/en/stable/api/satpy.multiscene.html#satpy.multiscene.MultiScene.blend) to combine your time series data. This will return a a regular Scene object that has been expanded in time:

In [ ]:
from satpy.multiscene import timeseries
bscn=mscn.blend(blend_function=timeseries)
bscn['C02']

A nice way to visualize your time series data is to create an animation, which we will do below:

In [ ]:
#Create the MultiScene
mscn = MultiScene.from_files(glob(abi_path+'OR*-RadM1*G16*.nc'), reader='abi_l1b')
#Load what you want
mscn.load(['C02'])
#Animate it
mscn.save_animation('Ian.mp4', fps=5)

This is what the above animation looks like:

In [ ]:
from IPython.display import Video
Video('Ian.mp4', width=480, height=480)

---